# **랭그래프**(**LangGraph**)

## 랭그래프란(LangGraph)

- **LangGraph**는 LangChain 생태계에서 개발된 **그래프구조 기반 워크플로우 및 에이전트 개발을 위한 런타임 프레임워크**이다.
- LangGraph는 선형적 흐름이 아닌 **비선형적인 복잡한 흐름 구조의 AI 에이전트 애플리케이션을 견고하고 재현 가능하게**(**build reliable**) 구성할 수 있도록 설계되었다.
- LangGraph의 가장 큰 특징은 **사이클**(**순환 구조**)을 지원한다는 점이다. 이는 특정 조건에 따라 이전 단계로 돌아가거나 다른 경로를 선택할 수 있음을 의미한다. 예를 들어, 검색 결과가 만족스럽지 않을 때 다시 검색을 수행하거나, 답변의 품질이 낮을 때 재생성을 요청하는 등의 복잡한 로직을 구현할 수 있다.

> **그래프**(**Graph**) **구조**
> - 데이터를 저장하고 조직화하는 자료구조 중 하나로 **데이터 간의 관계를 중심으로 정보를 표현하는 방식**의 자료구조다.
> - 데이터를 나타내는 **Node**(**노드**)와 Node와 Node를 연결하여 관계를 나타내는 **Edge**(**엣지**)로 데이터를 표현한다.
> - **트리**(**Tree**)는 루트를 중심으로 부모–자식(1:N) 위계를 구조적으로 강제하는 **계층적 관계의 자료구조**이다.
> - **그래프**(**Graph**)는 Node 간의 다대다(N:N) 관계를 edge 객체로 저장·연산에서 직접 다루는 비계층적 자료구조이며 수평적 관계까지 표현할 수 있다.


## LangGraph 핵심개념

- **그래프 기반 실행**: 애플리케이션의 실행 흐름을 **노드**(**node**)와 **엣지**(**edge**)로 구성된 그래프로 표현하여 분기, 병렬, 반복 구조를 자연스럽게 다룸.
- **상태 유지(stateful):** 실행 중인 에이전트의 상태(context)를 런타임이 관리하며 중단/재개 시에도 상태가 유지되도록 한다.
- **장기 실행(durable workflows):** 서버 재시작, 장시간 실행 등 프로덕션 조건에서도 상태를 유지하며 워크플로우를 이어갈 수 있는 기능 포함하고 있다.
- **멀티 에이전트 및 상호작용:** 여러 AI 에이전트가 협력하여 복잡한 작업을 수행할 수 있는 환경을 제공한다.




## 랭체인(LangChain)과의 관계

- *Langgraph는 LangChain의 기능을 확장하여 만들어진 프레임워크이다.* 
  - LangGraph는 LangChain에서 제공하는 여러 구성 요소(모델 호출, 도구 등)를 **low level에서 제어**할 수 있게 확장한 프레임워크이다.
    - LangGraph는 개발자가 모델 호출과 도구 실행 등의 순서, 조건에 따른 분기등을 직접 구성할 수 있다.
  - 필요 시 LangChain 에이전트를 LangGraph 워크플로우내의 Node로 사용할 수 있는 **상호 보완적 관계**이다.


### 비교

| 비교 항목            | **LangChain**                         | **LangGraph**                                     |  
| -------------------- | ------------------------------------- | ------------------------------------------------- | 
| 실행 흐름            | **선형/순차 중심** 워크플로우         | **그래프 기반** 복잡한 흐름 (분기·루프·상태 흐름) |
| 상태 관리            | 기본 메모리/세션 중심                 | **지속 상태(durable)**, 런타임 저장·복원          |
| 추상화 수준          | `LECE`을 이용한**고수준(high-level)** API | **저수준(low-level)** 실행 제어               | 
| 적합한 용도          | 간단한 에이전트 & 워크플로우          | 복잡한 실행흐름의 멀티 에이전트, 상태 기반 시스템 등 |



# 랭그래프 기본 구조

## 노드(Node) 와 엣지(Edge)

랭그래프의 기본 구조는 **노드**(**Node)** 와 **엣지**(**Edge**) **노드**(**Node)**로 구성된다.

- **노드**(**Node**): 
  - 그래프 상의 처리 단위
  - 실제 작업을 수행하는 Python 함수들로, 각각의 노드는 특정 로직을 담당한다. 
  - 노드는 현재 상태를 입력으로 받아서 작업을 수행하고, 업데이트된 상태를 반환한다.
  - 예를 들어, LLM 호출, 문서 검색, 데이터 처리 등의 기능을 수행할 수 있다.

- **엣지**(**Edge**):
  -  노드 간의 연결을 정의하며, 다음에 어떤 노드를 실행할지 결정하는 역할을 한다. 
  -  엣지는 워크플로우의 **흐름을 제어**하고 조건부 라우팅을 통해 **분기, 반복**을 가능하게 한다.

## 상태(State)

- 상태(State)는 에이전트가 실행되는 동안 유지되는 메모리/상태다. 실행 중 모든 노드가 이 상태에 접근할 수 있으며, 작업 결과에 따라 상태를 업데이트할 수 있다. 
- 상태(State) Schema(구조)는 **TypedDict 또는 pydantic를 상속한 클래스로 정의한다. Schema에는  State의 속성(Field)들을 설계한다.
- Agent **실행 중의 상태(State)는 Dictionary로 관리**되며, 스키마(schema)에 정의된 필드를 key로 가진다. (**스키마에 없는 field는 상태에 추가되지 않는다.**)
각 Node는 state 값을 읽어 사용하거나, 새로운 값을 추가·수정·병합(update)하여 다음 노드로 전달함으로써 상태를 변경한다. 

- **상태(State)의 주요 역할**
    - **전체 워크플로우의 컨텍스트 유지**
        -  에이전트가 여러 단계를 실행하는 동안 처리 과정에서 필요한 모든 정보를 한 곳(상태-dict)에서 관리한다
    - **노드 간 정보 공유**
        -  한 노드의 실행 결과를 state 필드에 저장하고, 다른 노드가 이를 조회해 사용하는 기능이다. 이를 통해 그래프 흐름에서 중간 데이터와 관계를 공유·재사용할 수 있다
    - **진행 상황 추적**
        -  에이전트가 현재 실행 중이거나 다음에 실행될 노드가 무엇인지, 이미 완료된 태스크는 무엇인지, 그리고 사용자 승인(HITL) 같은 의사결정 정보까지 state의 필드로 기록해 두는 기능. 
        -  이 정보들을 기반으로 그래프 실행 흐름을 조율하고, 조건 분기나 다음 단계 실행을 관리·추적할 수 있게 한다.


## StateGraph

- StateGraph는 랭그래프의 핵심 클래스로, 상태 기반의 계산 그래프를 정의하는 역할을 한다. StateGraph는 에이전트의 수명 주기 전반에 걸쳐 상태가 유지되도록 하며, 그래프의 노드들이 이 상태를 업데이트할 수 있게 한다.
- StateGraph의 초기화 과정(객체생성시)에서는 상태 정의 스키마를 받아서, 실행 시 Node들이 실행 결과를 dict로 반환하면 기존 상태(state)에 병합, 업데이트하여 전체 노드들이 공유할 수있도록 관리 한다.

## 랭그래프에서 상태관리

랭그래프에서 상태 관리는 두 가지 방식으로 이루어진다:

1. **완전 재정의 방식**:
   - 노드가 Field에 새 값을 반환하여 기존 값을 대체하는 방식이다. (다른 값으로 변경)
2. **업데이트 방식**: 
   - 기존 list와 같은 자료구조 타입의 Field에 새로운 값을 추가하는 방식이다. 예를 들어, 메시지 리스트에 새로운 메시지를 추가할 때 사용된다.

# 랭그래프 구현

- **설치**
  - `pip install langgraph`
  
## 상태(State) 구조(schema) 정의

- 랭그래프를 구현하기 위해서는 먼저 State 클래스를 정의 한다.
  - State는 `TypedDict`, `pydantic.BaseModel`를 상속받아 정의한다.
  - class 변수로 Field들을 정의 한다.
    - type hint를 이용해 각 Field가 어떤 타입의 값을 가질 지 선언한다. 
    - 리스트 병합 같은 결합 규칙이 필요한 필드는 `Annotated[..., reducer]` 형태로 reducer 함수를 지정한다.
    - Reducer는 field가 가진 기존 값에 노드가 추가한 새 값을 어떻게 결합할지 정의한다 (예: 리스트 추가, 값 덮어쓰기, 사용자 정의 병합 등)
  - State 클래스는 그래프 실행 전반에서 Node들이 공유/업데이트하는 상태 데이터의 스키마이며, 실제 실행 시 상태는 Dictionary 인스턴스로 관리된다.

    ```python
      from typing import TypedDict
      class MyState(TypedDict):
          counter: int
          messages: list 
    ```

## 노드 정의

- 노드는 실제 작업을 수행하는 Python 함수로 정의된다. 각 노드 함수는 현재 상태를 입력으로 받아 처리 후 업데이트된 상태를 반환한다.

  ```python
  def increment_counter(state):    
      return {"counter": state["counter"] + 1} 
      
  def process_message(state):
      # 메시지 처리 로직    
      new_message = "처리된 메시지"    
      return {"messages": state["messages"] + [new_message]}
  ```


## 엣지 정의

- 엣지는 노드 간의 연결을 정의하며, `add_edge` 메서드를 사용하여 추가한다. 기본적인 엣지는 한 노드에서 다른 노드로의 직접적인 연결을 의미한다.

  ```python
  graph.add_edge("node1", "node2") 
  graph.add_edge("node2", "node3")
  ```

## 그래프 구성

- 노드와 엣지를 정의한 후에는 그래프를 구성하고 컴파일해야 한다. 
  - `StateGraph`를 생성하고 그래프에 노드들을 추가한다.
  - 각 노드들을 연결하여 엣지를 추가한다. 이 과정에서 시작점과 종료점을 설정한다.
  - 컴파일을 통해 그래프의 구조를 검증한다.

    ```python
    from langgraph.graph import START, END
    from langgraph.graph import StateGraph 

    # StateGraph 생성
    graph = StateGraph(MyState)

    # 노드 추가 
    graph.add_node("increment", increment_counter) 
    graph.add_node("process", process_message) 
    # 엣지 추가 
    graph.add_edge(START, "increment") 
    graph.add_edge("increment", "process") 
    graph.add_edge("process", END) 
    # 그래프 컴파일 
    app = graph.compile(
    ```


## 호출

- `invoke()` 와 `stream()` 을 이용해 호출한다.
  - 메소드 호출시 **초기 상태(Initial state)를 제공**하여 그래프를 실행한다. 
    - 그래프는 이 초기 상태부터 시작하여 정의된 노드들을 순차적으로 실행하며 상태를 업데이트한다.
  - **checkpointer 저장소(메모리)를 사용**할 경우 **thread-id를 RunnableConfig로 전달**한다.
    - **thread_id**는 LangGraph에서 **하나의 workflow(그래프) 실행 세션을 식별하기 위한 ID** 
    - 개별 실행(invoke()/stream() 호출)이 아닌 **전체 작업시작 ~ 종료**까지의 논리적 단위를 식별하는 ID이다. 
- **invoke(input, config)**
  - 그래프(워크플로우)를 한 번 실행하고 최종 State 결과만 반환한다.
    ```python
    result = app.invoke({"counter": 0, "messages": []}) 
    print(result)
    ```
- **stream(input, config)**
  - 그래프 실행 중 발생하는 이벤트를 실시간으로 yield(스트리밍)하여 반환한다. 이벤트는 보통 각 node(task) 종료를 말한다.
  - **stream_mode** 설정에 따라 yield 되는 값이 다르다.
    - `"values"`: 각 event 시점의 **전체 State 상태**(State Snapshot이라고한다.)를 yield
    - `"updates"`: 각 노드(task)가 반환한 값. `{"node이름": {key:value, ..}}`
    - `"messages"`: Node가 출력한 대화 메세지(Agent/LLM 이 출력한)만 yield.

    ```python
      for event in app.stream({"counter": 0, "messages": []}) 
          print(event)
    ```

# 다양한 엣지

## **기본 엣지**

- 기본 엣지(Normal Edge)는 한 노드에서 다른 노드로의 직접적인 연결을 의미한다. 이는 가장 단순한 형태의 연결로, 이전 노드의 작업이 완료되면 자동으로 다음 노드로 이동한다.
- 기본 엣지는 다음과 같은 특징을 가진다:
  - **고정된 흐름**: 항상 동일한 다음 노드로 이동한다      
  - **단순한 구조**: 복잡한 조건 없이 순차적으로 실행된다      
  - **예측 가능한 동작**: 실행 전에 전체 흐름을 예측할 수 있다    

## **조건부 엣지와 라우팅**

- 조건부 엣지(Conditional Edge)는 특정 조건에 따라 다음에 실행할 노드를 동적으로 결정하는 기능이다. 이는 랭그래프의 가장 강력한 기능 중 하나로, 복잡한 의사결정 로직을 구현할 수 있게 한다.
- 조건부 엣지를 구현하기 위해서는 세 가지 구성 요소가 필요하다:
    - **1. 소스 노드(Source Node)**
      -  조건 평가가 시작될 노드의 이름이다.
    - **2. 조건 함수(Condition Function)**
      - 현재 상태를 입력받아 문자열을 반환하는 함수로, 다음에 실행할 노드를 결정하는 로직을 포함한다. 
    - **3. 조건부 엣지 매핑(Conditional Edge Mapping)**
      -  조건 함수의 반환값과 다음 노드 이름을 매핑하는 딕셔너리이다.
    ```python
    def relevance_check(state):     
        # 관련성 체크 로직    
        if state["relevance_score"] > 0.8:        
            return "grounded"    
        else:        
            return "not_grounded" 
            
    graph.add_conditional_edges(
        "llm_answer",     # 소스 노드    
        relevance_check,  # 조건 함수    
        {        
            "grounded": END,  # 조건함수반환값:이동할 노드 (grounded 이면 END(종료) 노드로 이동)
            "not_grounded": "retrieve"  # not_grounded 이면 retrieve 노드로 이동
        } 
    )
    ```

## **반복 구조**

- LangGraph 핵심 장점 중 하나는 반복 구조(순환 구조)를 지원한다는 점이다. 
- 반복 구조를 통해 특정 조건이 만족될 때까지 Node 간 전이를 순환시키거나, interrupt 이후 재개 실행까지 포함해 흐름을 반복할 수 있게 한다. 
- 반복 구조의 활용 예시:
  - **검색 품질 개선**
    - 검색 결과가 만족스럽지 않을 때 쿼리를 재작성하여 다시 검색한다      
  - **답변 품질 향상**
    - 생성된 답변의 품질이 낮을 때 다시 생성하거나 개선한다      
  - **할루시네이션 방지**
    -  답변이 검색된 문서와 관련성이 낮을 때 재생성을 요청한다

# LangGraph에서 메모리 관리

- **LangGraph의 메모리 관리**는 Checkpointer 저장소에 **실행 스레드(session/thread)** 단위의 **State(상태) 스냅샷**(전체 State)을 저장하고, 이후 필요할 때 다시 불러와 **그래프 실행을 중단 없이 이어가거나(HITL 등으로 중단 되었다가 재실행되어도 상태가 유지되도록한다),  특정 시점으로 복원할 수 있도록 지원하는 역할**을 말한다.
- **Checkpointer**는 그래프 세션의 **생명 주기 전반에서 상태를 지속 유지할 수 있게 하는 기반 컴포넌트**이며, 이를 통해 이전 실행 지점에서 다음 노드 실행을 재개하거나 저장된 상태를 활용할 수 있다.
  - Checkpointer는 **상태 저장소로 상태 저장, 복원의 기능을 제공** 하는 컴포넌트이다.

## 체크포인터의 주요 기능

1. **상태 지속성(State Persistence)**
   - 그래프 실행 중 특정 시점의 State(dict)를 저장하고, 이후 동일 스레드에서 다시 로드하여 실행을 이어갈 수 있게 한다.
   - 예를들어 사람의 개입이 필요해 agent 실행을 일시 멈췄다 재실행할 때 멈추기 전 상태를 그대로 유지할 수 있게 한다.

2. **대화 기록 누적 및 유지(Message Accumulation)**
   - 대화형 에이전트의 경우, `messages` 같은 state의 리스트 필드는 **Reducer(결합 함수)** 를 사용해 이전 메시지와 새 메시지를 병합(예: append)하여 저장한다. 이 메세지는 개별 workflow 호출에 대한 처리가 진행 중에만 유지된다.
   - Checkpointer는 개별 처리가 아닌 **전체 작업 시작 ~ 종료 까지 여러번에 걸친 실행 중 발생한 대화내역을 모두 저장하여** 대화 히스토리 관리가 가능하도록 하여 LLM과 멀티턴 대화가 가능하게 한다.

3. **체크포인트 기반 상태 복원(Checkpoint-based Recovery)**
   - 오류 발생 시 **이미 저장된 체크포인트 지점으로 상태를 복원할 수 있도록 한다.** 복구는 개발자가 설계한 체크포인트 지점을 기준으로 수행된다.


## 구현 방식

1. 그래프를 컴파일(compile)할 때 `checkpointer` 객체를 전달하여 연결한다.
2. 그래프를 실행(invoke/stream)할 때 `RunnableConfig.configurable` 필드에 `thread_id`를 전달해, **해당 실행 세션이 사용할 메모리 공간(저장 영역)** 을 지정한다.
3. 동일 `thread_id`를 가진 실행은 **서로 독립된 메모리 공간에서 상태를 공유**하며, 다른 `thread_id` 세션과 상태가 섞이지 않는다.

- **예제**:

  ```python
  from langgraph.checkpoint.memory import MemorySaver

  # 1. 메모리 기반 Checkpointer 생성
  checkpointer = MemorySaver()

  # 2. 그래프 compile 단계에서 Checkpointer 연결
  graph = workflow.compile(checkpointer=checkpointer)

  # 3. 그래프 실행 시 사용할 저장 스레드 ID 지정
  config = {"configurable": {"thread_id": "user-123"}}

  # 4. invoke()/stream() 호출 할 때 checkpointer 에게 전달할 thread_id 를 전달한다.
  response = graph.invoke({"messages": ["안녕하세요"]}, config=config)
  print(response)
  ```


# Human In The Loop (HITL)

- **Human-In-The-Loop**(**HITL**)은 AI 시스템의 의사결정 과정에 사람이 직접 개입하여 **검증, 승인, 수정, 피드백**을 제공하는 설계 패턴이다.
- LangGraph에서의 HITL은 **그래프**(**에이전트 워크플로우**)**의 특정 노드를 실행 중단**(**pause**)하고, 그 지점에서 사람의 입력을 받아 **다음 실행 방향을 결정하거나 상태(State)를 변경**한 뒤 흐름을 이어가는 방식으로 동작한다.

## HITL의 핵심 특성

 특성              | 설명                                              |
| --------------- | --------------------------------------------------------------------------- |
| 워크플로우 일시정지      | interrupt(중단) 기능으로 그래프 실행을 특정 노드에서 멈춤           |
| 상태(State) 수정 가능 | 사람이 중간 결과를 보고 State 값을 변경할 수 있음                 |
| 승인/거절/대안 흐름 선택  | 다음 엣지(경로)를 사람이 결정할 수 있음                         |
| 체크포인트 기반 복구     | 체크포인트(checkpoint) 시스템으로 이전 실행 지점으로 되돌리거나 재실행 가능 |
| 멀티 턴 인터랙션       | 필요 시 여러 번 사람과 상호작용 가능                           |

## HITL 사용 시점(Usecase)
HITL은 **사람의 판단이 필요한 고위험(high-stakes) 순간**, **맥락 기반 검증이 중요한 경우**, 또는 **AI 단독으로는 신뢰를 완전히 담보하기 어려운 경우**에 사용한다.

1. **결과 승인**
   - AI가 생성한 답변을 사람이 확인 후 승인해야 다음 단계로 진행
   - 예: 이메일 발송 전 내용 확인, 회사 계좌로 송금하기 전 최종 승인, 프로덕션 코드 배포 전 리뷰.
2. **모호성 해결 및 명확화**
   - 사용자 요청이 모호할 때 명확히 하기 위해 확인
    - 예: "사용자가 '그거 예약해줘'라고 했는데, '그거'가 무엇인지 AI가 모를 때 사람에게 물어봄."
3. **데이터 보정(중간 결과물 수정)**
   - AI가 추출한 정보가 애매하거나 추가 데이터가 필요한 경우 사람이 직접 수정 
   - 예: AI가 작성한 블로그 초안을 사람이 검토하고 수정한 뒤, 다시 AI에게 발행을 맡길 때.
4. **정책/규제 준수 검증**
   - 예: 금융, 의료, 법률 등 도메인에서 사람 검토 필수
5. **도구 실행 전 확인**
   - 예: 이메일 발송, DB 삭제, 결제 실행 등 AI가 액션을 수행하기 전 사람 승인
6. **학습 피드백**
   - 예: 모델이 틀렸을 경우 사람이 직접 피드백을 주고 흐름 재조정

## 관련 주요 API

| API                          | 역할                                        | 사용 목적                                   |
| ---------------------------- | ------------------------------------------- | -----------------------------------------   |
| `interrupt()`                | 현재 그래프 실행을 중단하고 외부 입력을 대기| 사람의 검토/입력을 받기 위해 그래프를 멈춤  |
| `MemorySaver / Checkpointer` | 실행 지점을 저장하는 체크포인트 시스템      | 이전 상태로 되돌리거나 중단 지점부터 재실행 하기 위해 상태가 저장되어 있어야 한다.|
| `invoke()`/`stream()`        | 그래프 실행 시작 또는 재개                  | HITL 입력 후 다시 실행                      |
| `Command(resume=...)`        | 재개 명령                                   | interrupt로 멈춘 곳에 사람의 입력값을 전달하며 실행을 다시 시작함 |
| `graph.update_state()`       | 상태(State) 수정                            | interrupt 되어 있는 동안 사람이 직접 state를 수정 할 수있게 한다.|


## 실행을 중단하는 두가지 방식
- 노드에서 `interrupt()` 를 호출 하여 명시적으로 멈춘다. (Dynamic Interrupt)
- Graph 컴파일시 `interrupt_before` 설정으로 노드를 지정하여 특정 노드가 실행하기 직전에 무조건 멈추게 한다. (Static Interrupt)
- **최신 버전에서는 Dynamic Interrupt 방식을 선호한다.**